# Spectral Analysis for Modal Parameter Linear Estimate

## Setup
### Libraries

In [ ]:
# Install requirements
import sys
!$sys.executable -m pip install -r ../requirements.txt --upgrade

from IPython import display
display.clear_output()

In [ ]:
# Add parent directory to path
import os
sys.path.insert(0, os.path.abspath(".."))

In [ ]:
import sample
from tests import utils
import numpy as np
from matplotlib import pyplot as plt
import librosa.display

### Generate test audio

In [ ]:
fs = 44100
x = utils.test_audio(fs=fs)

librosa.display.waveplot(x, sr=fs, zorder=100)
plt.grid()
plt.gcf().set_size_inches([12, 6])
display.Audio(x, rate=fs)

## Sinusoidal Model

In [ ]:
from sample.sms import mm
sm = mm.ModalModel(
    max_n_sines=10,
    peak_threshold=-30,
    save_intermediate=True
).fit(x)

### STFT

In [ ]:
stft = np.array([mx for mx, _ in sm.intermediate_["stft"]]).T

librosa.display.specshow(stft, sr=fs, x_axis="time", y_axis="hz");
plt.ylim([0, 2000])
plt.gcf().set_size_inches([12, 6])

### Peak detection

In [ ]:
mx, px = sm.intermediate_["stft"][0]
f = fs * np.arange(mx.size) / sm.w_.size
ploc, pmag, pph = sm.intermediate_["peaks"][0]

ax = plt.subplot(121)
plt.fill_between(f, np.full(mx.shape, -120), mx, alpha=.1)
plt.plot(f, mx)
plt.scatter(ploc * fs / sm.w_.size, pmag, c="C0")
plt.ylim([-60, plt.ylim()[1]])
plt.grid()
plt.title("magnitude")

plt.subplot(122, sharex=ax)
plt.plot(f, px)
plt.scatter(ploc * fs / sm.w_.size, pph)
plt.ylim([np.min(px[f < 2000]), np.max(px[f < 2000])])
plt.grid()
plt.title("phase")
plt.xlim([0, 2000])
plt.gcf().set_size_inches([12, 6])

### Sine tracking

In [ ]:
from sample import plots
plots.sine_tracking_2d(sm)
plt.gcf().set_size_inches([12, 6])

In [ ]:
from sample import plots
plots.sine_tracking_3d(sm)
plt.gcf().set_size_inches([12, 6])